In [1]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
from nltk.stem.porter import PorterStemmer

/mnt/kerem/anaconda3/envs/main/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to /home/kerem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kerem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, encoding='latin-1')

# Preprocessing

In [ ]:
def clean_sentence(text):
    # Remove links
    text = re.sub(r"http\S+", "", text)
    # Remove @ symbols and patterns in general
    text = re.sub("@[\w]*", '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stop-words
    text_tokens = word_tokenize(text)
    text = ' '.join([word.lower() for word in text_tokens if not word in stopwords.words() and word.isalpha()])
    # Stemming
    #text = [stemmer.stem(i) for i in word_tokenize(text)]
    return text

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()
stemmer = PorterStemmer()
data[6] = data[5].parallel_apply(clean_sentence)
data.to_csv("tweets.csv", sep='\t', encoding='latin-1')

# Feature Extraction

In [2]:
data = pd.read_csv("tweets.csv", sep='\t', header=None, encoding='latin-1')

###  Tf-idf Vectors

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [ ]:
x_tfidf = vectorizer.fit_transform(data.iloc[:, 7])

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb

In [17]:
pipeline = Pipeline([
    ('bow',CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    #('classifier', RandomForestClassifier(max_depth=20, n_jobs=-1, verbose=1)),  # train on TF-IDF vectors
    #('classifier',svm.LinearSVC(class_weight='balanced', verbose=5)),
    #('classifier', xgb.XGBClassifier(objective="binary:logistic", n_jobs=-1))
    ('classifier', xgb.XGBClassifier(objective="multi:softprob",eval_metric="auc",num_class=2,n_jobs=-1,learning_rate=0.001,tree_method="hist",
                          gamma=3,reg_lambda=10,max_depth=10,max_delta_step=1,colsample_bytree=0.95,scale_pos_weight=10000,num_parallel_tree=8,booster="dart"))
])

In [ ]:
msg_train, msg_test, label_train, label_test = train_test_split(data[7], data[1], test_size=0.2)
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))